In [90]:
import pandas as pd
import numpy as np

In [91]:
xls_CDNB = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/CDNB_cleaned.xlsx')
df_NB = pd.read_excel(xls_CDNB)
xls_NLQ = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/NLQ_cleaned.xlsx')
df_NLQ = pd.read_excel(xls_NLQ)
df_IT = df_NB.append(df_NLQ,ignore_index=True,sort=False) 

In [92]:
df_IT = df_IT.drop(df_IT[(df_IT.RegisterBuyVolume.isnull())&(df_IT.BuyVolume == 0)].index)
df_IT = df_IT.drop(df_IT[(df_IT.RegisterSellVolume.isnull())&(df_IT.SellVolume == 0)].index)

In [93]:
xls_1220 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2012_2020.xlsx')
xls_11 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2011.xlsx')
xls_21 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2021.xlsx')

df1_1220 = pd.read_excel(xls_1220, '1')
df2_1220 = pd.read_excel(xls_1220, '2')
df3_1220 = pd.read_excel(xls_1220, '3')
df4_11 = pd.read_excel(xls_11)
df5_21 = pd.read_excel(xls_21)

#Merge all into df_1121
df_1121 = df1_1220.append([df2_1220,df3_1220,df4_11,df5_21],ignore_index=True,sort=False)
df_1121_dup = df_1121.copy()
df_1121_dup.rename(columns={"TradingDate":"RegisterDate"}, inplace=True)

#Calculate daily ret of securities
df_1121_dup['Pi_ret'] = df_1121_dup.groupby('StockCode')['CloseAdj'].pct_change(-1)

In [94]:
#Map 'Exchange' & 'KLCPGD' values using cols
cols =['StockCode','RegisterDate']
df_IT['Exchange'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Exchange'])
df_IT['KLCPLH'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['KLCPLH'])

#Calculate date difference
from datetime import datetime
A = [d.date() for d in df_IT['DateStartExpected']]
B = [d.date() for d in df_IT['RegisterDate']]
df_IT['Diff_days'] = np.busday_count(B, A)

#Recalculate percentage values
df_IT['RegisterVolumeBeforePercent'] = df_IT['RegisterVolumeBefore']/df_IT['KLCPLH']
df_IT['RegisterVolumeAfterPercent'] = df_IT['RegisterVolumeAfter']/df_IT['KLCPLH']
df_IT['VolumeBeforePercent'] = df_IT['VolumeBefore']/df_IT['KLCPLH']
df_IT['VolumeAfterPercent'] = df_IT['VolumeAfter']/df_IT['KLCPLH']

#Cleaning data: drop all percentage values>1
df_IT = df_IT[(df_IT['RegisterVolumeBeforePercent'] < 1) &
               (df_IT['RegisterVolumeAfterPercent'] < 1) &
               (df_IT['VolumeBeforePercent'] < 1) & (df_IT['VolumeAfterPercent']<1)]

In [95]:
#Match Securities Ret
for x in range(-250,0):
    df_IT['Ri_T1_' + str(x)] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols).groupby('StockCode')['Pi_ret'].shift(x))

df_IT['Ri_T1_0'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Pi_ret'])

for x in range(1,11):
    df_IT['Ri_T1_' + str(x)] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols).groupby('StockCode')['Pi_ret'].shift(x))

In [96]:
df_IT.to_excel(r'/Users/hongnhungvu/Desktop/preview_all.xlsx',index=False)

#Here I export the file to clear all stock prices that does not belong [t,t+n], the file above is [t,t+11) and n is in [0,10]
#I use filter in Excel and clear manually
#Since I don't know how to do it in Python

In [97]:
xls_IT_1 = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/preview_all.xlsx')
df_IT_1 = pd.read_excel(xls_IT_1)

#Import the file after handling in Excel

In [98]:
#Benchmark trading data
xls_index = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Index_2011_2021.xlsx')
df_index = pd.read_excel(xls_index)
df_index.rename(columns={"TradingDate":"RegisterDate","IndexCode":"Exchange"}, inplace=True)

df_index['Exchange'] = df_index['Exchange'].replace(['VNINDEX','HNXINDEX','UPCOMINDEX'],['VNI','HNX','UPCOM'])

df_VNI = df_index[df_index['Exchange']=='VNI'].copy()
df_HNX = df_index[df_index['Exchange']=='HNX'].copy()
df_UPCOM = df_index[df_index['Exchange']=='UPCOM'].copy()

#HOSE and VNI are the same, I just use VNI here since HOSE's index is called VNI

#Calculate daily ret of benchmarks
df_VNI['Rm_ret'] = df_VNI['CloseIndex'].pct_change(-1)
df_HNX['Rm_ret'] = df_HNX['CloseIndex'].pct_change(-1)
df_UPCOM['Rm_ret'] = df_UPCOM['CloseIndex'].pct_change(-1)

In [99]:
#Match Benchmark Ret HNX
df_IT_HNX = df_IT_1[df_IT_1['Exchange']=='HNX'].copy()

for x in range(-250,0):
    df_IT_HNX['Rm_T1_' + str(x)] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'].shift(x))

df_IT_HNX['Rm_T1_0'] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'])

for x in range(1,10):
    df_IT_HNX['Rm_T1_' + str(x)] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'].shift(x))

    
#Match Benchmark Ret VNI
df_IT_VNI = df_IT_1[df_IT_1['Exchange']=='HOSE'].copy()

for x in range(-250,0):
    df_IT_VNI['Rm_T1_' + str(x)] = df_IT_VNI.set_index('RegisterDate').index.map(df_VNI.set_index('RegisterDate')['Rm_ret'].shift(x))

df_IT_VNI['Rm_T1_0'] = df_IT_VNI.set_index('RegisterDate').index.map(df_VNI.set_index('RegisterDate')['Rm_ret'])

for x in range(1,10):
    df_IT_VNI['Rm_T1_' + str(x)] = df_IT_VNI.set_index('RegisterDate').index.map(df_VNI.set_index('RegisterDate')['Rm_ret'].shift(x))
   

#Match Benchmark Ret UPCOM
df_IT_UPCOM = df_IT_1[df_IT_1['Exchange']=='UPCOM'].copy()

for x in range(-250,0):
    df_IT_UPCOM['Rm_T1_' + str(x)] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'].shift(x))

df_IT_UPCOM['Rm_T1_0'] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'])

for x in range(1,10):
    df_IT_UPCOM['Rm_T1_' + str(x)] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'].shift(x))

merged_df = pd.concat([df_IT_VNI, df_IT_HNX, df_IT_UPCOM])
merged_df

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,Rm_T1_0,Rm_T1_1,Rm_T1_2,Rm_T1_3,Rm_T1_4,Rm_T1_5,Rm_T1_6,Rm_T1_7,Rm_T1_8,Rm_T1_9
0,JVC,2012-01-13,2012-02-04,2012-01-19,2012-02-29,2012-01-16,2012-02-03,GD CD noi bo,13310000,0.550000,...,0.017868,0.009991,0.002766,0.010589,0.029697,-0.001151,0.032011,0.007871,0.006959,0.028003
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,0.018272,...,-0.022646,-0.012348,0.007692,0.015796,0.007978,0.001957,0.017868,0.009991,0.002766,0.010589
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,0.032308,...,-0.003314,-0.022646,-0.012348,0.007692,0.015796,0.007978,0.001957,0.017868,0.009991,0.002766
3,MPC,2012-02-09,2012-02-16,2012-02-16,2012-03-16,2012-02-16,2012-02-16,GD CD noi bo,7800,0.000111,...,0.004542,-0.015484,-0.023406,0.011933,-0.007120,-0.002265,0.017124,0.026482,-0.007416,0.018252
4,MPC,2012-02-10,2012-02-16,2012-02-16,2012-03-16,2012-02-16,2012-02-16,GD CD noi bo,17166,0.000245,...,-0.015484,-0.023406,0.011933,-0.007120,-0.002265,0.017124,0.026482,-0.007416,0.018252,0.013097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11903,PVP,2020-11-18,2020-12-23,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,9543747,0.101233,...,-0.004082,0.002580,0.004693,0.004973,0.001050,-0.002397,-0.001502,0.004361,0.001647,0.013602
11906,TR1,2020-11-19,2020-12-23,2020-11-23,2020-12-17,2020-11-23,2020-12-17,GD cua nguoi lien quan,268355,0.058433,...,0.002580,0.004693,0.004973,0.001050,-0.002397,-0.001502,0.004361,0.001647,0.013602,0.011650
11918,VRG,2020-12-04,2020-12-25,2020-12-07,2021-01-05,2020-12-07,2020-12-23,GD cua nguoi lien quan,1414000,0.054605,...,-0.005940,0.001312,0.003493,0.004642,-0.007941,0.000146,0.009313,0.003749,0.009049,0.000569
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,0.002790,0.002318,0.020046,-0.004082,0.002580,0.004693,0.004973,0.001050,-0.002397,-0.001502


In [100]:
import statsmodels.api as sm

#Regress Ri on Rm

def regress(data):
    result_all=[]
    for i in range(len(data)):
        Y = list(data.iloc[i][32:253])
        X = list(data.iloc[i][292:513])
        X = sm.add_constant(X)
        result = sm.OLS(Y,X,missing='drop').fit()
        result_all.append(result.params)
    return result_all  

#Why slice [32:253] and [292:513]
#merged_df.columns.get_indexer(['Ri_T1_-250', 'Ri_T1_-30','Rm_T1_-250', 'Rm_T1_-30'])
#array([ 32, 252, 292, 512])

#Run for HOSE first
res_VNI = regress(df_IT_VNI)

In [101]:
#Insert alphas & betas into df_IT_VNI
df_IT_VNI['alpha'] = np.array(res_VNI)[:,0]
df_IT_VNI['beta'] = np.array(res_VNI)[:,1]

In [102]:
#Calculate sd of abnormal return of each stock in each event in period (-250,-30)

def sd_ar(data):
    sd_ar_all=[]
    for i in range(len(data)):
        R_i = list(data.iloc[i][32:253])
        R_m = list(data.iloc[i][292:513])
        ar = [s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(R_i,R_m)]
        sd_ar_all.append([np.nanstd(ar)])
    return sd_ar_all

sd_VNI = sd_ar(df_IT_VNI)

In [103]:
#Insert sd of abnormal return into df_IT_VNI
df_IT_VNI['sd_ar'] = np.array(sd_VNI)[:,0]

In [104]:
#Calculate cumulative abnormal return (-30,-1)

def cal_car_before(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][252:282])
        r_m = list(data.iloc[i][512:542])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [252:282] and [512:542]
#df_IT_VNI.columns.get_indexer(['Ri_T1_-30', 'Ri_T1_-1','Rm_T1_-30', 'Rm_T1_-1'])
#array([252, 281, 512, 541])

cal_before_VNI = cal_car_before(df_IT_VNI)

In [105]:
#Insert car before into df_IT_VNI
df_IT_VNI['car_before'] = np.array(cal_before_VNI)

In [106]:
df_IT_VNI['ar_T0'] = df_IT_VNI['Ri_T1_0'] - (df_IT_VNI['Rm_T1_0']*df_IT_VNI['beta']+df_IT_VNI['alpha'])

def cal_car_after(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][283:292])
        r_m = list(data.iloc[i][543:552])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [283:292] and [543:552]
#df_IT_VNI.columns.get_indexer(['Ri_T1_1', 'Ri_T1_9','Rm_T1_1', 'Rm_T1_9'])
#array([283, 291, 543, 551])

cal_after = cal_car_after(df_IT_VNI)

In [107]:
#Insert car after into df_IT_VNI
df_IT_VNI['car_after'] = np.array(cal_after)

df_IT_VNI['car_t_t+n'] = df_IT_VNI['car_after'] + df_IT_VNI['ar_T0']

In [108]:
#Calculate t-statistics (separately for each event, but I don't use this result)
df_IT_VNI['t-stat_before'] = df_IT_VNI['car_before']/(df_IT_VNI['sd_ar']*(30**0.5))
df_IT_VNI['t-stat_eventdate'] = df_IT_VNI['ar_T0']/df_IT_VNI['sd_ar']
df_IT_VNI['t-stat_after'] = df_IT_VNI['car_after']/(df_IT_VNI['sd_ar']*((df_IT_VNI['Diff_days']-1)**0.5))
df_IT_VNI['t-stat_t_t+n'] = df_IT_VNI['car_t_t+n']/(df_IT_VNI['sd_ar']*(df_IT_VNI['Diff_days']**0.5))
df_IT_VNI

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,beta,sd_ar,car_before,ar_T0,car_after,car_t_t+n,t-stat_before,t-stat_eventdate,t-stat_after,t-stat_t_t+n
0,JVC,2012-01-13,2012-02-04,2012-01-19,2012-02-29,2012-01-16,2012-02-03,GD CD noi bo,13310000,5.500000e-01,...,1.272007,0.149195,0.262166,-0.010915,-0.198114,-0.209028,0.320819,-0.073159,-0.766654,-0.700521
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,1.827250e-02,...,0.873415,0.018895,0.135031,0.021753,-0.003751,0.018003,1.304754,1.151278,-0.140361,0.550086
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,3.230815e-02,...,1.048919,0.015433,0.104821,0.030377,0.020168,0.050545,1.240065,1.968318,0.924085,1.890921
3,MPC,2012-02-09,2012-02-16,2012-02-16,2012-03-16,2012-02-16,2012-02-16,GD CD noi bo,7800,1.114286e-04,...,0.833024,0.022558,0.014642,-0.018838,-0.068183,-0.087020,0.118507,-0.835064,-1.511242,-1.725147
4,MPC,2012-02-10,2012-02-16,2012-02-16,2012-03-16,2012-02-16,2012-02-16,GD CD noi bo,17166,2.452286e-04,...,0.832069,0.022551,0.035153,0.002964,-0.071018,-0.068054,0.284593,0.131441,-1.818156,-1.508848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11931,VDP,2020-12-23,2020-12-29,2020-12-28,2020-12-29,2020-12-28,2020-12-28,GD cua nguoi lien quan,1826264,1.137894e-01,...,0.388968,0.026840,-0.107862,0.000672,-0.003968,-0.003295,-0.733722,0.025051,-0.104526,-0.070882
11932,PPC,2020-12-17,2020-12-30,2020-12-22,2021-01-19,2020-12-22,2020-12-25,GD cua nguoi lien quan,77396260,2.414008e-01,...,0.786193,0.013602,-0.018956,0.009613,-0.012425,-0.002812,-0.254429,0.706743,-0.645928,-0.119360
11933,PDR,2020-12-10,2020-12-30,2020-12-14,2021-01-12,2020-12-14,2020-12-28,GD cua nguoi lien quan,92106,2.324912e-04,...,0.220897,0.019955,-0.002598,-0.017152,-0.006070,-0.023222,-0.023766,-0.859573,-0.304171,-0.822891
11934,VCB,2020-12-15,2020-12-30,2020-12-18,2021-01-15,2020-12-28,2020-12-29,GD cua nguoi lien quan,2523,6.802597e-07,...,1.026902,0.011936,0.014556,-0.012699,-0.014702,-0.027401,0.222664,-1.063933,-0.871009,-1.325438


In [109]:
df_VNI_buy = pd.DataFrame(df_IT_VNI[(df_IT_VNI['RegisterSellVolume'].isnull())])
df_VNI_sell = pd.DataFrame(df_IT_VNI[(df_IT_VNI['RegisterBuyVolume'].isnull())])

In [110]:
df_IT_VNI['StockCode'].nunique()

419

In [111]:
#Form groups of events for t-test & Wilcoxon test

import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

df_buy_2 = df_VNI_buy[(df_VNI_buy['Diff_days']==2)]
df_sell_2 = df_VNI_sell[(df_VNI_sell['Diff_days']==2)]

df_buy_3 = df_VNI_buy[(df_VNI_buy['Diff_days']==3)]
df_sell_3 = df_VNI_sell[(df_VNI_sell['Diff_days']==3)]

df_buy_4 = df_VNI_buy[(df_VNI_buy['Diff_days']==4)]
df_sell_4 = df_VNI_sell[(df_VNI_sell['Diff_days']==4)]

df_buy_5 = df_VNI_buy[(df_VNI_buy['Diff_days']==5)]
df_sell_5 = df_VNI_sell[(df_VNI_sell['Diff_days']==5)]

In [112]:
#Wilcoxon tests
w2b, p2b = wilcoxon(df_buy_2["car_t_t+n"])
w2b, p2b, norm.ppf(p2b/2), df_buy_2["car_t_t+n"].median()

(20576.0, 2.8117355457079402e-05, -4.188201714931374, 0.005951037100448537)

In [113]:
w2s, p2s = wilcoxon(df_sell_2["car_t_t+n"])
w2s, p2s, norm.ppf(p2s/2), df_sell_2["car_t_t+n"].median()

(68195.0, 0.3285640388359071, -0.9770103289550282, -0.002112881787818543)

In [114]:
w3b, p3b = wilcoxon(df_buy_3["car_t_t+n"])
w3b, p3b, norm.ppf(p3b/2), df_buy_3["car_t_t+n"].median()

(799279.0, 3.1387097585918036e-14, -7.5924671894605815, 0.004412995075385951)

In [115]:
w3s, p3s = wilcoxon(df_sell_3["car_t_t+n"])
w3s, p3s, norm.ppf(p3s/2), df_sell_3["car_t_t+n"].median()

(1583064.0,
 3.9510510280939785e-13,
 -7.2572274224143944,
 -0.0061437091013183285)

In [116]:
w4b, p4b = wilcoxon(df_buy_4["car_t_t+n"])
w4b, p4b, norm.ppf(p4b/2), df_buy_4["car_t_t+n"].median()

(68387.0, 7.470189976116405e-07, -4.948723182015137, 0.006640250097040458)

In [117]:
w4s, p4s = wilcoxon(df_sell_4["car_t_t+n"])
w4s, p4s, norm.ppf(p4s/2), df_sell_4["car_t_t+n"].median()

(124368.0, 0.00036540272496161924, -3.563887418687802, -0.006712968548458486)

In [118]:
w5b, p5b = wilcoxon(df_buy_5["car_t_t+n"])
w5b, p5b, norm.ppf(p5b/2), df_buy_5["car_t_t+n"].median()

(3750.0, 0.026283177877233232, -2.2220018899404668, 0.007712945798945186)

In [119]:
w5s, p5s = wilcoxon(df_sell_5["car_t_t+n"])
w5s, p5s, norm.ppf(p5s/2), df_sell_5["car_t_t+n"].median()

(6221.0, 0.027547106314603668, -2.2036764527108303, -0.0089761199821385)

In [120]:
#t-test
import scipy.integrate 

from scipy import stats

tstat_VNI_2_buy, pval_VNI_2_buy = scipy.stats.ttest_1samp(df_buy_2['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_2_buy, pval_VNI_2_buy, df_buy_2['car_t_t+n'].mean(), df_buy_2['car_t_t+n'].std(), len(df_buy_2)

(3.5144413527943152,
 0.0005016269918687713,
 0.007352701880720926,
 0.038235246165517565,
 334)

In [121]:
tstat_VNI_2_sell, pval_VNI_2_sell = scipy.stats.ttest_1samp(df_sell_2['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_2_sell, pval_VNI_2_sell, df_sell_2['car_t_t+n'].mean(), df_sell_2['car_t_t+n'].std(), len(df_sell_2)

(0.31887509095598926,
 0.7499459217917067,
 0.0007297736565861349,
 0.05293519017163029,
 535)

In [122]:
tstat_VNI_3_buy, pval_VNI_3_buy = scipy.stats.ttest_1samp(df_buy_3['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_3_buy, pval_VNI_3_buy, df_buy_3['car_t_t+n'].mean(), df_buy_3['car_t_t+n'].std(), len(df_buy_3)

(6.805701379891468,
 1.3257180796031993e-11,
 0.008249262023883062,
 0.05412585705044227,
 1994)

In [123]:
tstat_VNI_3_sell, pval_VNI_3_sell = scipy.stats.ttest_1samp(df_sell_3['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_3_sell, pval_VNI_3_sell, df_sell_3['car_t_t+n'].mean(), df_sell_3['car_t_t+n'].std(), len(df_sell_3)

(-4.837662833355039,
 1.3863581477602316e-06,
 -0.005036922229009227,
 0.05455076223046831,
 2745)

In [124]:
tstat_VNI_4_buy, pval_VNI_4_buy = scipy.stats.ttest_1samp(df_buy_4['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_4_buy, pval_VNI_4_buy, df_buy_4['car_t_t+n'].mean(), df_buy_4['car_t_t+n'].std(), len(df_buy_4)

(4.745726211197075,
 2.6038826415091496e-06,
 0.012130194086992336,
 0.06245284625528437,
 597)

In [125]:
tstat_VNI_4_sell, pval_VNI_4_sell = scipy.stats.ttest_1samp(df_sell_4['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_4_sell, pval_VNI_4_sell, df_sell_4['car_t_t+n'].mean(), df_sell_4['car_t_t+n'].std(), len(df_sell_4)

(-0.32780462024669643,
 0.7431493198362904,
 -0.0010368543253336665,
 0.08742776023057665,
 764)

In [126]:
tstat_VNI_5_buy, pval_VNI_5_buy = scipy.stats.ttest_1samp(df_buy_5['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_5_buy, pval_VNI_5_buy, df_buy_5['car_t_t+n'].mean(), df_buy_5['car_t_t+n'].std(), len(df_buy_5)

(2.1249148088438736,
 0.03538891020736341,
 0.016249677752118392,
 0.08983442097193524,
 138)

In [127]:
tstat_VNI_5_sell, pval_VNI_5_sell = scipy.stats.ttest_1samp(df_sell_5['car_t_t+n'],0,alternative='two-sided')
tstat_VNI_5_sell, pval_VNI_5_sell, df_sell_5['car_t_t+n'].mean(), df_sell_5['car_t_t+n'].std(), len(df_sell_5)

(-1.7199008511847163,
 0.08722848809129566,
 -0.009745822469488631,
 0.07496078206569537,
 175)